# **Step1_AI면접관 Agent v1.0**

## **1. 환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project_genai)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### (2) 라이브러리

In [2]:
!pip install -r /content/drive/MyDrive/project_genai/requirements.txt -q

### (3) OpenAI API Key 확인
* api_key.txt 파일에 다음의 키를 등록하세요.
    * OPENAI_API_KEY
    * NGROK_AUTHTOKEN

In [3]:
import os

def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path = '/content/drive/MyDrive/project_genai/'
# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

In [4]:
print(os.environ['OPENAI_API_KEY'][:30])

sk-proj-eb2vf2EeDektdAURPzCxaM


## **2. App.py**

* 아래 코드에, Step1 혹은 고도화 된 Step2 파일 코드를 붙인다.
    * 라이브러리
    * 함수들과 그래프
* Gradio 코드는 그대로 사용하거나 일부 수정 가능

In [5]:
%%writefile app.py


####### 여러분의 함수와 클래스를 모드 여기에 붙여 넣읍시다. #######
## 1. 라이브러리 로딩 ---------------------------------------------
import pandas as pd
import numpy as np
import os
import ast
import fitz  # PyMuPDF
from docx import Document
import random
import openai
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from typing import Annotated, Literal, Sequence, TypedDict

from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langgraph.graph import StateGraph, START, END

## ---------------- 1단계 : 사전준비 ----------------------

# 1) 파일 입력 --------------------
def extract_text_from_file(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()

    if ext == ".pdf":
        doc = fitz.open(file_path)
        text = "\n".join(page.get_text() for page in doc)
        doc.close()
        return text

    elif ext == ".docx":
        doc = Document(file_path)
        return "\n".join(p.text for p in doc.paragraphs if p.text.strip())

    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 허용됩니다.")

# 2) State 선언 --------------------
from typing import TypedDict, List, Dict

class InterviewState(TypedDict):
    # 고정 정보
    resume_text: str
    resume_summary: str
    resume_keywords: List[str]
    question_strategy: Dict[str, Dict]

    # 인터뷰 로그
    current_question: str
    current_answer: str
    current_strategy: str
    conversation: List[Dict[str, str]]
    evaluation : List[Dict[str, str]]
    next_step : str

    #
    retry_count: int
    question_index: int
    strategy_analysis : Dict[str, str]

# 3) resume 분석 --------------------
def analyze_resume(state: InterviewState) -> InterviewState:
    # 여기에 코드를 완성합니다.
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
    prompt = PromptTemplate(
        template='''
        당신은 면접을 미리 준비하는 면접관입니다.
        당신은 다음 자기소개서를 보고 5줄로 요약해서 미리 준비하고자 합니다.
        결과는 자기소개서 확인 후 5줄 내외로 요약합니다.
        ---
        자기소개서 : {resume_text}
        ---
        ''',
        input_variables=["resume_text"]
    )
    resume_summary = (prompt | llm).invoke(state["resume_text"]).content

    prompt = PromptTemplate(
        template='''
        당신은 면접을 미리 준비하는 면접관입니다.
        다음 자기소개서를 보고 중요하다고 판단되는 키워드 10개 가량 생각하려고 합니다.
        결과는 각 키워드를 , 로 구분하여 나열합니다.
        답변 예시 : [... , ... , ...]
        ---
        자기소개서 : {resume_text}
        ---
        ''',
        input_variables=["resume_text"]
    )
    resume_keywords = (prompt | llm | CommaSeparatedListOutputParser()).invoke(state["resume_text"])


    # return 코드는 제공합니다.
    return {
        **state,
        "resume_summary": resume_summary,
        "resume_keywords": resume_keywords,
    }


# 4) 질문 전략 수립 --------------------

def generate_question_strategy(state: InterviewState) -> InterviewState:
    # 여기에 코드를 완성합니다.
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

    prompt = PromptTemplate(
        template='''
        당신은 면접 준비를 하는 면접관입니다.
        당신은 요약된 자기소개서와 질문 키워드를 받아 어떤 질문을 해야할지 질문 전략을 생각하려 합니다.
        질문 전략의 분야는 다음과 같습니다. : ["경력 및 경험", "동기 및 커뮤니케이션","논리적 사고"]
        답변은 아래와 같은 구조를 따릅니다.
        {{
            "경력 및 경험" : {{
                "질문 방향성" : "...",
                "예시 질문" : "..."
             }},
             "동기 및 커뮤니케이션" : {{
                "질문 방향성" : "...",
                "예시 질문" : "..."
             }},
             "논리적 사고":{{
                "질문 방향성" : "...",
                "예시 질문" : "..."
             }}
        }}

        ---
        요약된 자기 소개서 : {resume_summary}
        ---
        질문 키워드 : {resume_keywords}
        ---
        ''',
        input_variables=["resume_summary", "resume_keywords"]
    )
    resume_summary = state['resume_summary']
    resume_keywords = state['resume_keywords']

    response = (prompt | llm | StrOutputParser()).invoke(
        {
            "resume_summary": resume_summary,
            "resume_keywords": resume_keywords
        }
    )
    import json
    strategy_dict = json.loads(response)

    # return 코드는 제공합니다.
    return {
        **state,
        "question_strategy": strategy_dict
    }


# 5) 1단계 하나로 묶기 --------------------
from docx import Document as DocxDocument
from langchain.schema import Document as LC_Document

def make_vectorDB_get_retriever(file_path: str):
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        from langchain.document_loaders import PyMuPDFLoader
        loader = PyMuPDFLoader(file_path)
        raw_docs = loader.load()
        documents = [LC_Document(page_content=doc.page_content, metadata=doc.metadata) for doc in raw_docs]
    elif ext == ".docx":
        doc = DocxDocument(file_path)
        resume_text = "\n".join(p.text for p in doc.paragraphs if p.text.strip())
        documents = [LC_Document(page_content=resume_text)]
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 허용됩니다.")

    #임베딩
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

    db = Chroma.from_documents( documents,embeddings, persist_directory='_DB')

    #리트리버 구축
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

    #리트리버 툴
    from langchain.tools.retriever import create_retriever_tool

    retriever_tool = create_retriever_tool(
        retriever,
        'retriever_DB',
        '''The Retriever searches and returns the information on your resume.''' #<-------= 역할 부여
    )

    return [retriever,retriever_tool,db]


def preProcessing_Interview(file_path: str) -> InterviewState:
    # 여기에 코드를 완성합니다.

    # 이력서 추출

    resume_text = retriever.get_relevant_documents("이력서")[0].page_content # "이력서"와 같은 검색어를 사용

    # 초기 상태 구성
    initial_state: InterviewState = {
        "resume_text": resume_text,
        "resume_summary": '',
        "resume_keywords": [],
        "question_strategy": {},

        "current_question": '',
        "current_answer": '',
        "current_strategy": '',
        "conversation": [],
        "evaluation": [],
        "next_step" : '',

        "retry_count" : 0,
        "question_index" : 0,
        "strategy_analysis" : {}
    }
    state = initial_state
    # 요약추가 특징 추가
    state = analyze_resume(initial_state)
    # 면접질문 추출
    state = generate_question_strategy(state)
    # 첫번째 질문 세팅
    selected_question = state['question_strategy']['경력 및 경험']['예시 질문']
    # return 코드는 제공합니다.
    return {
            **state,
            "current_question": selected_question,
            "current_strategy": "경력 및 경험"
            }

## ---------------- 2단계 : 면접 Agent ----------------------

# 1) 답변 입력 --------------------
def update_current_answer(state: InterviewState, user_answer: str) -> InterviewState:
    return {
        **state,
        "current_answer": user_answer.strip()
    }

# 2) 답변 평가 --------------------
def evaluate_answer(state: InterviewState) -> InterviewState:
    # 여기에 코드를 완성합니다.
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

    prompt = PromptTemplate(
        template = '''
        당신은 인공지능 면접관입니다.
        다음 정보를 보고 면접자의 질문이 적절한지 판단합니다.

        당신은 내용을 보고 '질문의 연관성'과 '답변의 구체성'을  바탕으로 평가 등급을 4가지 분류로 나뉩니다.
        그후 평가 이유 및 보완점을 서술합니다.
        보완점을 구체적으로 서술한다.

        질문의 연관성 평가 기준

        1.매우 우수
        질문의 의도를 완벽히 이해하고 답변하며, 직무·회사·상황 맥락까지 자연스럽게 연결한다. 질문의 핵심뿐 아니라 확장된 의미까지 파악하여 심화된 내용을 말할 수 있다.

        2.우수
        질문의 의도를 잘 이해했고, 핵심 내용을 빠짐없이 언급한다. 다만 추가적인 배경이나 심화 내용은 다소 부족할 수 있다.

        3.보통
        질문과 부분적으로만 연결되며, 핵심 내용 중 일부가 빠지거나 표면적인 이해에 머무른다. 약간 질문에서 벗어난 부분이 느껴질 수 있다.

        4.미흡
        질문과 거의 관련 없거나 질문 의도를 잘못 이해했다. 답변이 엉뚱한 방향으로 흘러가거나 주제를 벗어난다. 대답이 짧거나 질문에 전혀 대답하지 않거나 ‘모르겠습니다’, ‘열심히 하겠습니다’ 같은 형식적 표현에 그친다.

        답변의 구체성 평가 기준

        1.매우 우수
        구체적 경험, 데이터, 행동, 결과, 배운 점까지 매우 상세히 언급한다.

        2.우수
        구체적 사례, 경험, 행동, 결과 중 2~3가지를 포함한다. 답변이 비교적 명확하고 구체적이지만, 일부 세부 정보는 부족할 수 있다.

       3. 보통
        일부 구체적인 내용은 있으나 예시나 경험이 부족하거나, 경험은 언급했으나 결과나 배운 점이 빠졌다. 다소 추상적인 답변이 섞여 있다.

        4.미흡
        매우 추상적이고 원론적인 답변으로, 개인 경험이나 구체적 사례가 거의 없다. 대답이 짧거나 질문에 전혀 대답하지 않거나 ‘모르겠습니다’, ‘열심히 하겠습니다’,“최선을 다하겠습니다” 같은 형식적 표현에 그친다.

        ---
        이력서 요약 : {resume_summary}

        이력서 키워드 : {resume_keywords}

        면접 질문 전략 : {current_strategy}

        면접관의 질문 : {current_question}

        면접자의 답변 : {current_answer}

        ---
        주의: 반드시 아래 형식으로 JSON 하나만 출력하세요. 예:
        {{
        "질문의 연관성": "매우 우수",
        "답변의 구체성": "미흡"
        "이유 및 보완점": "답변이 질문 의도에 부합하지만 구체적 사례가 부족합니다."
        }}
        그 외 다른 문장은 절대 출력하지 마세요.
        특히, '안녕하세요', '파이팅', '잘 부탁드립니다', '모르겠습니다' 같은 인사말이나 형식적, 질문과 연관 없는 답변은 반드시 '질문의 연관성: 미흡', '답변의 구체성: 미흡'으로 평가하고, 그 이유를 명시하세요.

        ''',
        input_variables = ["resume_summary", "resume_keywords", "current_strategy", "current_question", "current_answer"]
    )
    response = (prompt | llm | StrOutputParser()).invoke({
        "resume_summary": state["resume_summary"],
        "resume_keywords": state["resume_keywords"],
        "current_strategy": state["current_strategy"],
        "current_question": state["current_question"],
        "current_answer": state["current_answer"]
    })

    import json
    evaluation = json.loads(response)

    conversation = state['conversation']
    conversation.append({

        "question": state["current_question"],
        "answer": state["current_answer"],
        "evaluation": evaluation,
        "strategy" : list(state['question_strategy'].keys())[state['question_index']],
        #"strategy" : state["current_strategy"],
    })
    state["conversation"] = conversation

    # return 코드는 제공합니다.
    return {
        **state,
        "evaluation": evaluation
    }

# 3) 인터뷰 진행 검토 --------------------
def decide_next_step(state: InterviewState) -> InterviewState:
    MAX_TOTAL_QA_COUNT = 5
    MAX_RETRY_COUNT = 2  # 재질문 최대 카운트 설정

    # 이미 진행된 질문 수
    total_turns = len(state["conversation"])
    question_index = state["question_index"]
    retry_count = state["retry_count"]

    # 전략 목록
    strategies = list(state['question_strategy'].keys())  # 전략 목록을 state에서 받음

    # 최근 평가
    eval_rel = state["evaluation"].get("질문의 연관성", "")
    eval_detail = state["evaluation"].get("답변의 구체성", "")

    # 1. 전체 질문/답변이 5번 넘었으면 종료
    if total_turns >= MAX_TOTAL_QA_COUNT:
        next_step = "end"

    # 2. 최근 평가에 '미흡'이 있으면 현재 전략에서 재질문
    elif eval_rel == "미흡" or eval_detail == "미흡":
        if retry_count >= MAX_RETRY_COUNT:
            # 재질문이 2번 이상이면 다음 전략으로 전환
            if question_index < len(strategies):
                next_step = "additional_question"  # 기존 전략에서 재질문
                retry_count += 1  # 재질문 카운트 증가
            else:
                next_step = "additional_question"
                retry_count = 0  # 재질문 카운트 초기화
                question_index += 1  # 전략을 다음으로 전환

        else:
            next_step = "additional_question"
            retry_count += 1  # 재질문 카운트 증가

    # 3. 그 외에는 추가 질문으로 전환 (재질문)
    else:
        if question_index < len(strategies):
            next_step = "additional_question"
            question_index += 1  # 질문 인덱스 증가
            retry_count = 0  # 재질문 카운트 초기화
        else:
            next_step = "end"  # 전략의 끝

    # question_index 범위 초과 방지
    if question_index >= len(strategies):
        next_step = "end"

    #-------------------------------------- 디버깅 코드
    # if next_step =="end" :
    #     print("종료")
    # else :
    #     print(f"다음 인덱스 : {question_index}")
    #     print(f"누적 질문 카운트 : {retry_count}")
    #     print(f"다음 전략 : {state['question_strategy'][strategies[question_index]]}")
    #-------------------------------------- 디버깅 코드

    return {
        **state,
        "next_step": next_step,
        "retry_count": retry_count,
        "question_index": question_index,  # 질문 인덱스를 state에 반영
    }


# 4) 질문 생성 --------------------
def generate_question(state: InterviewState) -> InterviewState:
    llm = ChatOpenAI(temperature=0.2, model_name="gpt-4o-mini")

    # 평가 기반 전략 전환 여부 결정
    # 현재 전략에 대한 평가 리스트 가져오기
    eval_1 = state["evaluation"].get("질문의 연관성", [])
    eval_2 = state["evaluation"].get("답변의 구체성", [])
    latest_eval = eval_1 + eval_2

    # latest_eval = state["evaluation"].get("질문의 연관성", []), state["evaluation"].get("답변의 구체성", [])
    latest_eval = list(latest_eval)

    # 검색 기준: current_strategy + resume_keywords
    search_query = f"{state['current_strategy']} {state['resume_keywords']}"

    # 유사 질문 검색 (최신 방식: invoke 사용)
    similar_docs = retriever.invoke(search_query)

    # 최대 3개까지 추출
    similar_questions = [doc.page_content.strip() for doc in similar_docs[:3]]


    prompt = PromptTemplate(
        template = '''
        당신은 인공지능 면접관입니다.

        아래의 유사 질문을 참고하여, 면접자의 상황에 맞는 새로운 질문을 1개 생성해 주세요.
        기존 질문을 사용하지 말고, 유사 질문을 바탕으로 새롭고 심화된 질문을 만드세요.

        --- 유사한 질문 예시 ---
        {similar_questions}

        --- 이력 정보 및 상황 ---
        이력서 요약: {resume_summary}
        이력서 키워드: {resume_keywords}
        면접 질문 전략: {current_strategy}
        이전 질문: {current_question}
        면접자의 답변: {current_answer}
        면접관의 평가: {evaluation}
        ''',
        input_variables=[
            "resume_summary",
            "resume_keywords",
            "current_strategy",
            "current_question",
            "current_answer",
            "evaluation",
            "similar_questions"
        ]
    )

    # LLM 호출
    response = (prompt | llm | StrOutputParser()).invoke({
        "resume_summary": state["resume_summary"],
        "resume_keywords": state["resume_keywords"],
        "current_strategy": state["current_strategy"],
        "current_question": list(state['question_strategy'].keys())[state['question_index']],
        #"current_question": state["current_question"],
        "current_answer": state["current_answer"],
        "evaluation": state["evaluation"],
        "similar_questions": similar_questions
    })
    # print("----------- response -----------")
    # print(response)

    # 질문 단위로 저장
    doc = LC_Document(
        page_content=response.strip(),
        metadata={"strategy": state["current_strategy"], "source": "LLM"}
    )

    # 벡터 DB에 추가
    vectorDB.add_documents([doc])

    # 변경사항 저장 #Chroma 0.4 이상에서는 persist_directory를 지정하기만 하면 문서가 자동 저장됩니다.
    #vectorDB.persist()

    return {
        **state,
        "current_question": response.strip(),
        "current_answer": ""
    }

# 5) 인터뷰 피드백 보고서 --------------------
def summarize_interview(state: InterviewState) -> InterviewState:
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.3)

    strategies = list(state.get("question_strategy", {}).keys())
    conversations = state.get("conversation", [])

    strategy_analysis = {}
    all_qa_pairs = []
    all_evaluations = []

    # 전략별 분석을 진행하는 부분
    for strategy in strategies:
        # print(f"디버깅: 현재 전략 - {strategy}")

        # 해당 전략에 맞는 질문과 답변 데이터 추출
        qa_pairs = [
            f"Q: {c['question']}\nA: {c['answer']}"
            for c in conversations
            if c.get("strategy") == strategy
        ]
        eval_texts = [
            f"질문 연관성: {e['evaluation']['질문의 연관성']}, 답변 구체성: {e['evaluation']['답변의 구체성']}"
            for e in conversations
            if e.get("strategy") == strategy
        ]

        # 디버깅: qa_pairs와 eval_texts 출력
        # print(f"디버깅: {strategy} 전략에 대한 qa_pairs: {qa_pairs}")
        # print(f"디버깅: {strategy} 전략에 대한 eval_texts: {eval_texts}")

        # 해당 전략에 대한 질문과 평가를 바탕으로 분석 진행
        qa_text = "\n\n".join(qa_pairs)
        evaluation_text = "\n".join(eval_texts)

        prompt = PromptTemplate(
            template="""다음은 특정 전략에 대한 면접 질문과 지원자의 대답, 그리고 평가 내용입니다.
이 정보를 기반으로 해당 전략에서의 **답변 스타일**, **강점**, **약점**을 각각 한 문단으로 정리해주세요.

면접 전략: {strategy}

질문 및 답변:
{qa_text}

평가:
{evaluation_text}

출력 형식:
답변 스타일:
(문단)

강점:
(문단)

약점:
(문단)
""",
            input_variables=["strategy", "qa_text", "evaluation_text"]
        )

        full_prompt = prompt.format(
            strategy=strategy,
            qa_text=qa_text,
            evaluation_text=evaluation_text
        )

        response = (llm | StrOutputParser()).invoke(full_prompt)
        response_split = response.split('\n\n')

        style = response_split[0].replace("답변 스타일:", "").strip()
        strengths = response_split[1].replace("강점:", "").strip()
        weaknesses = response_split[2].replace("약점:", "").strip()

        strategy_analysis[strategy] = {
            "답변 스타일": style,
            "강점": strengths,
            "약점": weaknesses
        }

        all_qa_pairs.extend(qa_pairs)
        all_evaluations.extend(eval_texts)

    # 전체적인 면접 분석
    overall_prompt = PromptTemplate(
        template="""다음은 면접 전체에 대한 면접 질문과 지원자의 대답, 그리고 평가 내용입니다.
이 정보를 기반으로 전체적인 **인상**, **강점**, **보완할 점**을 각각 한 문단으로 정리해주세요.

질문 및 답변:
{qa_text}

평가:
{evaluation_text}

출력 형식:
전체적인 인상:
(문단)

강점:
(문단)

보완할 점:
(문단)
""",
        input_variables=["qa_text", "evaluation_text"]
    )

    all_qa_text = "\n\n".join(all_qa_pairs)
    all_evaluation_text = "\n".join(all_evaluations)

    overall_full_prompt = overall_prompt.format(
        qa_text=all_qa_text,
        evaluation_text=all_evaluation_text
    )

    overall_response = (llm | StrOutputParser()).invoke(overall_full_prompt)
    overall_response_split = overall_response.split('\n\n')

    overall_impression = overall_response_split[0].replace("전체적인 인상:", "").strip()
    overall_strengths = overall_response_split[1].replace("강점:", "").strip()
    overall_weaknesses = overall_response_split[2].replace("보완할 점:", "").strip()

    # 최종 상태에 추가
    updated_state = {
        **state,
        "strategy_analysis": strategy_analysis,
        "overall_summary": {
            "전체적인 인상": overall_impression,
            "강점": overall_strengths,
            "보완할 점": overall_weaknesses
        }
    }

    # 🔹 여기서 출력 기능을 추가 🔹
    print("🔹 전략별 분석 🔹")
    for strategy, analysis in strategy_analysis.items():
        print(f"\n▶ 전략: {strategy}")
        print(f"[답변 스타일]\n{analysis.get('답변 스타일', '')}\n")
        print(f"[강점]\n{analysis.get('강점', '')}\n")
        print(f"[약점]\n{analysis.get('약점', '')}\n")

    print("\n🔹 전체 요약 🔹")
    print(f"\n[전체적인 인상]\n{overall_impression}\n")
    print(f"[강점]\n{overall_strengths}\n")
    print(f"[보완할 점]\n{overall_weaknesses}\n")

    return updated_state


# 6) Agent --------------------
# 분기 판단 함수
def route_next(state: InterviewState) -> Literal["generate_question", "summarize_interview"]:
    return "summarize_interview" if state["next_step"] == "end" else "generate_question"

# 그래프 정의 시작
builder = StateGraph(InterviewState)

# 노드 추가
builder.add_node(evaluate_answer, "evaluate_answer")
builder.add_node(decide_next_step, "decide_next_step")
builder.add_node(generate_question, "generate_question")
builder.add_node(summarize_interview, "summarize_interview")

# 노드 연결
builder.add_edge(START, "evaluate_answer")
builder.add_edge("evaluate_answer", "decide_next_step")
builder.add_conditional_edges("decide_next_step", route_next,{
    "summarize_interview": "summarize_interview",
    "generate_question": "generate_question"
})# 출력 이름과 노드이름이 같으면 생략가능

builder.add_edge("generate_question", END)
builder.add_edge("summarize_interview", END)

# 컴파일
graph = builder.compile()
#-------------------------------------------------------------------


########### 다음 코드는 제공되는 gradio 코드 입니다.################

import gradio as gr
import tempfile

# 세션 상태 초기화 함수
def initialize_state():
    return {
        "state": None,
        "interview_started": False,
        "interview_ended": False,
        "chat_history": []
    }

# 파일 업로드 후 인터뷰 초기화
def upload_and_initialize(file_obj, session_state):
    if file_obj is None:
        return session_state, "파일을 업로드해주세요."

    # Gradio는 file_obj.name 이 파일 경로야
    file_path = file_obj.name

    # 인터뷰 사전 처리
    state = preProcessing_Interview(file_path)
    session_state["state"] = state
    session_state["interview_started"] = True

    # 첫 질문 저장
    first_question = state["current_question"]
    session_state["chat_history"].append(["🤖 AI 면접관", first_question])

    return session_state, session_state["chat_history"]

# 답변 처리 및 다음 질문 생성
def chat_interview(user_input, session_state):
    if not session_state["interview_started"]:
        return session_state, "먼저 이력서를 업로드하고 인터뷰를 시작하세요."

    # (1) 사용자 답변 저장
    session_state["chat_history"].append(["🙋‍♂️ 지원자", user_input])
    session_state["state"] = update_current_answer(session_state["state"], user_input)

    # (2) Agent 실행 (평가 및 다음 질문 or 종료)
    session_state["state"] = graph.invoke(session_state["state"])

    # ✅ 만약 인터뷰가 종료된 경우, summarize_interview 직접 호출
    if session_state["state"]["next_step"] == "end":
        session_state["state"] = summarize_interview(session_state["state"])

    # (3) 종료 여부 판단
    if session_state["state"]["next_step"] == "end":
        session_state["interview_ended"] = True

        final_summary = "✅ 인터뷰가 종료되었습니다!\n\n"

        # 1. 질문/답변/평가 요약
        for i, turn in enumerate(session_state["state"]["conversation"]):
            final_summary += f"\n**[질문 {i+1}]** {turn['question']}\n**[답변 {i+1}]** {turn['answer']}\n"
            eval_result = turn.get("evaluation", {})
            final_summary += f"_평가 - 질문 연관성: {eval_result.get('질문의 연관성', '-')}, 답변 구체성: {eval_result.get('답변의 구체성', '-')}_\n"

        # 2. 전략별 분석 요약 추가
        final_summary += "\n\n🔹 전략별 분석 🔹\n"
        for strategy, analysis in session_state["state"].get("strategy_analysis", {}).items():
            final_summary += f"\n▶ 전략: {strategy}\n"
            final_summary += f"[답변 스타일]\n{analysis.get('답변 스타일', '')}\n"
            final_summary += f"[강점]\n{analysis.get('강점', '')}\n"
            final_summary += f"[약점]\n{analysis.get('약점', '')}\n"

        # 3. 전체 요약 추가
        overall = session_state["state"].get("overall_summary", {})
        final_summary += "\n\n🔹 전체 요약 🔹\n"
        final_summary += f"[전체적인 인상]\n{overall.get('전체적인 인상', '')}\n"
        final_summary += f"[강점]\n{overall.get('강점', '')}\n"
        final_summary += f"[보완할 점]\n{overall.get('보완할 점', '')}\n"

        session_state["chat_history"].append(["🤖 AI 면접관", final_summary])
        return session_state, session_state["chat_history"], gr.update(value="")

    else:
        next_question = session_state["state"]["current_question"]
        session_state["chat_history"].append(["🤖 AI 면접관", next_question])
        return session_state, session_state["chat_history"], gr.update(value="")

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    session_state = gr.State(initialize_state())

    gr.Markdown("# 🤖 AI 면접관 \n이력서를 업로드하고 인터뷰를 시작하세요!")

    with gr.Row():
        file_input = gr.File(label="이력서 업로드 (PDF 또는 DOCX)")
        upload_btn = gr.Button("인터뷰 시작")

    chatbot = gr.Chatbot()
    user_input = gr.Textbox(show_label=False, placeholder="답변을 입력하고 Enter를 누르세요.")

    upload_btn.click(upload_and_initialize, inputs=[file_input, session_state], outputs=[session_state, chatbot])
    user_input.submit(chat_interview, inputs=[user_input, session_state], outputs=[session_state, chatbot])
    user_input.submit(lambda: "", None, user_input)

# 실행
# 파일 입력
path ='/content/drive/MyDrive/project_genai/'
file_path = path + 'Resume_sample.docx'

retriever ,retriever_tool,vectorDB= make_vectorDB_get_retriever(file_path)
state = preProcessing_Interview(file_path)

demo.launch(share=True)

Overwriting app.py


## **3. 실행**

In [ ]:
!python app.py

/content/app.py:766: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()
/content/app.py:182: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
/content/app.py:206: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resume_